<img src='https://www.icos-cp.eu/sites/default/files/2017-11/ICOS_CP_logo.png' width=400 align=right>

# ICOS Carbon Portal Python Libraries 

This example uses a foundational library called `icoscp_core` which can be used to access time-series ICOS data that are <i>previewable</i> in the ICOS Data Portal. "Previewable" means that it is possible to visualize the data variables in the preview plot. The library can also be used to access (meta-)data from [ICOS Cities](https://citydata.icos-cp.eu/portal/) and [SITES](https://data.fieldsites.se/portal/) data repositories. 

General information on all ICOS Carbon Portal Python libraries can be found on our [help pages](https://icos-carbon-portal.github.io/pylib/). 

Documentation of the `icoscp_core` library, including information on running it locally, can also be found on [PyPI.org](https://pypi.org/project/icoscp_core/).

Note that for running this example locally, authentication is required (see the `how_to_authenticate.ipynb` notebook).

# Example: Access ICOS data and plot with data from PANGEA

This example highlights the possibility of using two independent datasets. One is loaded from the ICOS Carbon Portal's ocean domain and contains data from a boat traveling from Europe to Brazil. The second dataset contains similar data collected from another boat traveling from Europe to South America.

Originally, this example was created as a demonstrator in support of “Integrating data and analysis technologies within leading environmental research infrastructures: Challenges and approaches,” Huber et al., 2021, https://doi.org/10.1016/j.ecoinf.2021.101245. Since then, small updates in the graphic presentations have been made.


## Import libraries

In [ ]:
# Import some standard libraries
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import folium
import os
import cartopy
cartopy.config['data_dir'] = '/data/project/cartopy/'
import cartopy.crs as ccrs
import cartopy.feature as feat
import numpy as np
import pandas as pd

# Import ICOS and PANGAEA library
from icoscp_core.icos import meta, data
from pangaeapy import PanDataSet

## Load data from ICOS and PANGAEA

In [ ]:
icosmeta = meta.get_dobj_meta('https://meta.icos-cp.eu/objects/xgu4rfCmqvXb4w1wGGD6mYsB')
# please use data.batch_get_columns_as_arrays when fetching data for uniform lists of data objects
icosdata = data.get_columns_as_arrays(icosmeta)
icosdata_frame = pd.DataFrame(icosdata)

pandata = PanDataSet('10.1594/PANGAEA.889516')
pandata_frame=pandata.data

## Let's have a look at the returned data

In [ ]:
icosdata_frame.head(3)

In [ ]:
pandata_frame.head(3)

## Plot showing water temperatures


### Dates and temperatures

In [ ]:
plt.figure(figsize=(10,6))
ax = plt.axes()
icosdata_frame.plot(x='TIMESTAMP', y='Temp [degC]', label ='ICOS data',ax=ax,linestyle='None', marker='o',grid='on',legend=True)
pandata_frame.plot(x='Date/Time', y='Temp',label ='PANGEA data',ax=ax, linestyle='None', marker='o',grid='on')
plt.show()

### Dates, temperatures, and latitude

In [ ]:

# Combine the temperature data from both datasets to find the global min and max
min_temp = min(icosdata_frame['Temp [degC]'].min(), pandata_frame['Temp'].min())
max_temp = max(icosdata_frame['Temp [degC]'].max(), pandata_frame['Temp'].max())

plt.figure(figsize=(10, 6))
ax = plt.axes()

# Plotting ICOS data with Latitude on y-axis and color based on Temp [degC]
icos_scatter = ax.scatter(
    x=icosdata_frame['TIMESTAMP'], 
    y=icosdata_frame['Latitude'], 
    c=icosdata_frame['Temp [degC]'], 
    cmap='coolwarm', 
    vmin=min_temp, 
    vmax=max_temp, 
    label='ICOS data (right)', 
    marker='o'
)

# Plotting PANGEA data with Latitude on y-axis and color based on Temp
pangea_scatter = ax.scatter(
    x=pandata_frame['Date/Time'], 
    y=pandata_frame['Latitude'], 
    c=pandata_frame['Temp'], 
    cmap='coolwarm',  # Use the same colormap
    vmin=min_temp, 
    vmax=max_temp, 
    label='PANGEA data (left)', 
    marker='o'
)

# Add a single colorbar for both scatter plots
cbar = plt.colorbar(icos_scatter, ax=ax)
cbar.set_label('Temperature [degC]')

# Adding grid, legend, and title
plt.grid(True)
plt.legend()
plt.title('Water Temperature vs Latitude')
plt.xlabel('Timestamp')
plt.ylabel('Latitude')

plt.show()



### Interactive map 

Hover over the points on the map to see more information about each observation. You can switch between different basemaps.

In [ ]:

# Combine the temperature data from both datasets to find the global min and max
min_temp = min(icosdata_frame['Temp [degC]'].min(), pandata_frame['Temp'].min())
max_temp = max(icosdata_frame['Temp [degC]'].max(), pandata_frame['Temp'].max())

# Create a colormap using matplotlib's 'coolwarm'
cmap = plt.cm.get_cmap('coolwarm')

# Normalize the temperature values to the range [0, 1] for the colormap
norm = mcolors.Normalize(vmin=min_temp, vmax=max_temp)

# Create the map
myMap = folium.Map(location=[20, 0], zoom_start=2)
folium.TileLayer('openstreetmap').add_to(myMap)
folium.TileLayer(tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', attr = 'Esri',show=False, name='ESRI satellite').add_to(myMap)  # use folium to add alternative tiles

# Add ICOS data with colored markers based on temperature
for lat, lon, temp, time in zip(icosdata_frame['Latitude'], icosdata_frame['Longitude'], icosdata_frame['Temp [degC]'], icosdata_frame['TIMESTAMP']):
    folium.CircleMarker(
        location=[lat, lon],
        radius=2,
        color=mcolors.to_hex(cmap(norm(temp))),
        fill=True,
        fill_color=mcolors.to_hex(cmap(norm(temp))),
        fill_opacity=0.7,
        tooltip=f"ICOS data<br>{time}<br>{temp:.2f} °C"
    ).add_to(myMap)

# Add PANGEA data with colored markers based on temperature
for lat, lon, temp, time in zip(pandata_frame['Latitude'], pandata_frame['Longitude'], pandata_frame['Temp'], pandata_frame['Date/Time']):
    folium.CircleMarker(
        location=[lat, lon],
        radius=2,
        color=mcolors.to_hex(cmap(norm(temp))),
        fill=True,
        fill_color=mcolors.to_hex(cmap(norm(temp))),
        fill_opacity=0.7,
        tooltip=f"PANGEA data<br>{time}<br>{temp:.2f} °C"
    ).add_to(myMap)

# Add a layer control to toggle between the tile layers
folium.LayerControl().add_to(myMap)

# Display the map
myMap


### Static map


In [ ]:

# Combine the temperature data from both datasets to find the global min and max
min_temp = min(pandata_frame['Temp'].min(), icosdata_frame['Temp [degC]'].min())
max_temp = max(pandata_frame['Temp'].max(), icosdata_frame['Temp [degC]'].max())

# Combine latitude and longitude data to find the global extent
min_lat = min(pandata_frame['Latitude'].min(), icosdata_frame['Latitude'].min())
max_lat = max(pandata_frame['Latitude'].max(), icosdata_frame['Latitude'].max())
min_lon = min(pandata_frame['Longitude'].min(), icosdata_frame['Longitude'].min())
max_lon = max(pandata_frame['Longitude'].max(), icosdata_frame['Longitude'].max())

# Create the map with the dynamic extent
proj = ccrs.Miller()
plt.figure(dpi=150)
ax = plt.axes(projection=proj)
ax.set_extent([min_lon, max_lon, min_lat, max_lat], crs=ccrs.PlateCarree())
ax.stock_img()
land_50m = feat.NaturalEarthFeature('physical', 'land', '50m', edgecolor='grey', facecolor=feat.COLORS['land'])
ax.add_feature(land_50m)
ax.coastlines()

# Scatter plots with consistent color scale
scatter_pan = ax.scatter(
    pandata_frame['Longitude'], pandata_frame['Latitude'],
    c=pandata_frame['Temp'], s=0.2, transform=ccrs.PlateCarree(),
    cmap='coolwarm', vmin=min_temp, vmax=max_temp
)

scatter_icos = ax.scatter(
    icosdata_frame['Longitude'], icosdata_frame['Latitude'],
    c=icosdata_frame['Temp [degC]'], s=0.3, transform=ccrs.PlateCarree(),
    cmap='coolwarm', vmin=min_temp, vmax=max_temp
)

# Add a colorbar to represent temperature
cbar = plt.colorbar(scatter_pan, ax=ax, orientation='vertical', pad=0.02)
cbar.set_label('Temperature [°C]', fontsize=8)  # Change label font size
cbar.ax.tick_params(labelsize=8)  # Change tick label font size

# Optionally, save the figure
# plt.savefig('name_of_plot.png')

plt.show()


## Cite the data set

In [ ]:
print('ICOS:')
print(icosmeta.references.citationString)

print('PANGEA:')
print(pandata.citation)